In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import count
import json


In [2]:
# === 参数设置 ===
builder_dir = r"E:\droid_part"  # TFDS 数据目录（含 dataset_info.json / features.json / tfrecord-*）
output_dir = r"F:\PangYe\shit4"  # 输出目录
split = "train"
max_samples = 20400 # None 表示全部样本；设为整数表示只处理这么多
SAMPLE_PAD = 5  # 样本编号位数
STEP_PAD = 5  # 步编号位数

# 三个需要导出的图像键
IMAGE_KEYS = ["exterior_image_1_left", "exterior_image_2_left", "wrist_image_left"]

# OpenCV 用于更快的PNG编码（pip install opencv-python）
PNG_COMPRESSION = 3  # PNG 压缩等级（0～9）

# 并发写盘线程数（NVMe 可适当开大）
NUM_WRITE_WORKERS = 2

# tf.data 加速设置：非确定性（乱序）+ 预取
ALLOW_NON_DETERMINISTIC = False

In [3]:
# === 创建输出目录 ===
OUTPUT_ROOT = Path(output_dir)
# 创建 OUTPUT_ROOT 路径（如果它不存在）
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

In [4]:
# === 加载数据集 ===
# 使用 builder_from_directory 加载数据集
builder = tfds.builder_from_directory(builder_dir)
info = builder.info
ds = builder.as_dataset(split=split)
print(info)


tfds.core.DatasetInfo(
    name='r2d2_faceblur',
    full_name='r2d2_faceblur/1.4.0',
    description="""
    
    """,
    homepage='https://www.tensorflow.org/datasets/catalog/r2d2_faceblur',
    data_dir='E:\\droid_part',
    file_format=tfrecord,
    download_size=Unknown size,
    dataset_size=1.67 TiB,
    features=FeaturesDict({
        'episode_metadata': FeaturesDict({
            'file_path': string,
            'recording_folderpath': string,
        }),
        'steps': Dataset({
            'action': Tensor(shape=(7,), dtype=float64, description=Robot action, consists of [6x joint velocities, 1x gripper position].),
            'action_dict': FeaturesDict({
                'cartesian_position': Tensor(shape=(6,), dtype=float64, description=Commanded Cartesian position),
                'cartesian_velocity': Tensor(shape=(6,), dtype=float64, description=Commanded Cartesian velocity),
                'gripper_position': Tensor(shape=(1,), dtype=float64, description=Commanded

In [5]:
# === 工具函数 ===
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p

def to_pil_uint8(arr):
    """确保是 uint8 的 HWC 数组再转 PIL.Image"""
    if isinstance(arr, np.ndarray):
        a = arr
    else:
        # 兼容 EagerTensor
        a = np.array(arr)
    if a.dtype != np.uint8:
        a = a.astype(np.uint8)
    return Image.fromarray(a)

def decode_str(x):
    """兼容 bytes / EagerTensor / numpy 的字符串转换为 Python str"""
    if x is None:
        return ""
    if isinstance(x, bytes):
        return x.decode("utf-8", errors="ignore")
    try:
        # numpy scalar of bytes or str
        if isinstance(x, np.ndarray) and x.ndim == 0:
            x = x.item()
    except Exception:
        pass
    if hasattr(x, "numpy"):
        x = x.numpy()
        if isinstance(x, bytes):
            return x.decode("utf-8", errors="ignore")
    return str(x)

def write_text(path: Path, text: str, encoding="utf-8"):
    path.write_text(text, encoding=encoding)

def save_png_lossless(img_np: np.ndarray, out_path: Path, compression: int = PNG_COMPRESSION):
    """使用 OpenCV 写 PNG（始终无损）"""
    out_path.parent.mkdir(parents=True, exist_ok=True)
    if img_np.ndim == 3 and img_np.shape[2] == 3:
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    else:
        img_bgr = img_np
    cv2.imwrite(str(out_path), img_bgr, [cv2.IMWRITE_PNG_COMPRESSION, int(compression)])


In [6]:
# === tf.data 加速设置 ===
options = tf.data.Options()
options.experimental_deterministic = True
ds = ds.with_options(options).prefetch(tf.data.AUTOTUNE)


## 分开

In [8]:
# ==== 主处理循环（PNG + 并发写盘；每样本使用本地步号命名 step1/step2/...） ====
from tqdm import tqdm
# 不再需要 from itertools import count
max_samples = 20
# 有损 JPEG 质量（数值越低越小，常用 75~95）
JPEG_QUALITY = 90

def save_jpeg_lossy(img_np: np.ndarray, out_path: Path, quality: int = JPEG_QUALITY):
    """用 OpenCV 写 JPEG（有损）。TFDS 图像为 RGB，这里转 BGR 再写。"""
    out_path.parent.mkdir(parents=True, exist_ok=True)
    if img_np.ndim == 3 and img_np.shape[2] == 3:
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    else:
        img_bgr = img_np
    cv2.imwrite(str(out_path), img_bgr, [cv2.IMWRITE_JPEG_QUALITY, int(quality)])

total_examples = info.splits[split].num_examples or None
processed_samples = 0

executor = ThreadPoolExecutor(max_workers=NUM_WRITE_WORKERS)
futures = []

pbar = tqdm(total=total_examples, desc="Processing episodes (JPEG, per-episode step naming)", unit="ep")

def submit_jpg(img_np, folder_path: Path, fname: str):
    futures.append(executor.submit(save_jpeg_lossy, img_np, folder_path / fname, JPEG_QUALITY))

for sample_index, example in enumerate(ds, start=1):

    # 限制最多处理多少个样本
    if max_samples is not None and processed_samples >= max_samples:
        pbar.write(f"[INFO] Reached max_samples={max_samples}, stopping early.")
        break

    # 取 steps 子数据集（tf.data.Dataset）
    if "steps" not in example:
        pbar.write(f"[WARN] sample {sample_index} has no 'steps'; skipped.")
        continue

    steps_ds = example["steps"]
    if not isinstance(steps_ds, tf.data.Dataset):
        pbar.write(f"[WARN] sample {sample_index} 'steps' is not a tf.data.Dataset; skipped.")
        continue

    # === 每个样本的目录结构 ===
    sample_id = f"id{processed_samples + 1:0{SAMPLE_PAD}d}"
    sample_dir = OUTPUT_ROOT / sample_id
    images_root = ensure_dir(sample_dir / "images")
    sub_dirs = {k: ensure_dir(images_root / k) for k in IMAGE_KEYS}

    wrote_lang = False
    step_count_this_episode = 0

    for t, step in enumerate(steps_ds):
        step_count_this_episode += 1

        # 将本 step 的张量转 numpy
        step_np = tf.nest.map_structure(lambda x: x.numpy(), step)

        # 第一条 language_instruction -> .txt（如无需可移除这段）
        if not wrote_lang:
            first_lang = decode_str(step_np.get("language_instruction", b""))
            write_text(sample_dir / "language_instruction.txt", first_lang)
            wrote_lang = True

        # 三路图像
        obs = step_np.get("observation", {})
        try:
            img1 = obs["exterior_image_1_left"]
            img2 = obs["exterior_image_2_left"]
            imgw = obs["wrist_image_left"]
        except KeyError as e:
            pbar.write(f"[WARN] sample {sample_index} step {t} missing image key {e}; skip.")
            continue

        # === 文件名从 0 开始 ===
        fname = f"{t}.jpg"   # 0.jpg, 1.jpg, 2.jpg, ...

        # 异步写盘（JPEG 有损）
        submit_jpg(img1, sub_dirs["exterior_image_1_left"], fname)
        submit_jpg(img2, sub_dirs["exterior_image_2_left"], fname)
        submit_jpg(imgw, sub_dirs["wrist_image_left"],   fname)

    if step_count_this_episode == 0:
        pbar.write(f"[WARN] sample {sample_index} has 0 steps; removing empty folder.")
        try:
            for k in IMAGE_KEYS:
                (images_root / k).rmdir()
            images_root.rmdir()
            (sample_dir / "language_instruction.txt").unlink(missing_ok=True)
            sample_dir.rmdir()
        except Exception:
            pass
        continue

    processed_samples += 1
    pbar.update(1)

# 等待所有写任务完成
for f in tqdm(as_completed(futures), total=len(futures), desc="Flushing JPEG writes", unit="img"):
    _ = f.result()
executor.shutdown(wait=True)

print(f"Done. Episodes processed: {processed_samples}")
print(f"Total JPG images submitted: {len(futures)}")
print(f"Output root: {OUTPUT_ROOT}")

Processing episodes (JPEG, per-episode step naming):   0%|          | 20/92233 [00:12<6:50:06,  3.75ep/s] 

[INFO] Reached max_samples=20, stopping early.


Flushing JPEG writes: 100%|██████████| 13851/13851 [00:12<00:00, 1144.08img/s]

Done. Episodes processed: 20
Total JPG images submitted: 13851
Output root: F:\PangYe\shit2


## 一个视频 跳过没有的

In [7]:
# ==== 主处理循环（每相机1个视频；无损优先 FFV1 -> 失败则退回 MJPG） ====
# 视频参数
FPS = 15  # 你可以改为真实采样率；不知道就填 15/20/30 都行
USE_LOSSLESS = True  # 无损优先

jsonl_path = OUTPUT_ROOT / "language_instructions.jsonl"
jsonl_path.parent.mkdir(parents=True, exist_ok=True)
jsonl_f = open(jsonl_path, "a", encoding="utf-8")

def _make_writer(out_path: Path, frame_size_hw, fps=FPS, lossless=USE_LOSSLESS):
    H, W = frame_size_hw
    out_path.parent.mkdir(parents=True, exist_ok=True)

    # 尝试 FFV1 (无损) —— 需要 OpenCV 是 FFmpeg 后端并支持该编码
    if lossless:
        try:
            fourcc = cv2.VideoWriter_fourcc(*'FFV1')
            vw = cv2.VideoWriter(str(out_path.with_suffix('.mkv')), fourcc, fps, (W, H))
            if vw.isOpened():
                return vw, '.mkv'
        except Exception:
            pass

    # 回退：MJPG（avi）—— 不是严格无损，但兼容性最好
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    vw = cv2.VideoWriter(str(out_path.with_suffix('.avi')), fourcc, fps, (W, H))
    if not vw.isOpened():
        raise RuntimeError(f"Cannot open VideoWriter for {out_path}")
    return vw, '.avi'

total_examples = info.splits[split].num_examples or None
# 统计：按“遍历过的样本数”截断；sample_id 用这个计数，不跳号
seen_samples = 0
wrote_videos = 0

pbar = tqdm(total=(max_samples if max_samples is not None else total_examples),
            desc="Per-camera videos (only if any text)", unit="ep")

for sample_index, example in enumerate(ds, start=1):
    # 先计入“已遍历样本数”
    seen_samples += 1
    # 按遍历顺序给 sample_id，不跳号
    sample_id = f"id{seen_samples:0{SAMPLE_PAD}d}"

    # 达到用户设定的上限就停止（按“遍历过的样本数”）
    if max_samples is not None and seen_samples > max_samples:
        pbar.write(f"[INFO] Reached max_samples={max_samples} (by seen samples), stopping.")
        break

    # 没有 steps 的样本直接跳过（ID 仍然占用）
    if "steps" not in example:
        pbar.write(f"[WARN] {sample_id}: no 'steps'; skipped.")
        pbar.update(1)
        continue

    steps_ds = example["steps"]
    if not isinstance(steps_ds, tf.data.Dataset):
        pbar.write(f"[WARN] {sample_id}: 'steps' not tf.data.Dataset; skipped.")
        pbar.update(1)
        continue

    # 先窥视第一步：读取三条文本并判断
    first_step_np = None
    for _first in steps_ds.take(1):
        first_step_np = tf.nest.map_structure(lambda x: x.numpy(), _first)
    if first_step_np is None:
        pbar.write(f"[WARN] {sample_id}: 0 steps; skipped.")
        pbar.update(1)
        continue

    instr1 = decode_str(first_step_np.get("language_instruction", b"")).strip()
    instr2 = decode_str(first_step_np.get("language_instruction_2", b"")).strip()
    instr3 = decode_str(first_step_np.get("language_instruction_3", b"")).strip()
    has_any_text = any([bool(instr1), bool(instr2), bool(instr3)])

    if not has_any_text:
        # 三条都空：完全跳过（不写 JSONL、不建视频）
        pbar.write(f"[SKIP] {sample_id}: all three instructions empty; no JSONL, no video.")
        pbar.update(1)
        continue

    # 有任意一条文本：写 JSONL（一行一个样本；缺的留空）
    rec = {
        "sample_id": sample_id,
        "instruction_1": instr1,
        "instruction_2": instr2,
        "instruction_3": instr3,
    }
    jsonl_f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    jsonl_f.flush()

    # 准备输出目录与 writer（仅现在才创建目录）
    sample_dir = OUTPUT_ROOT / sample_id
    ensure_dir(sample_dir)

    writers = {}     # key -> cv2.VideoWriter
    suffixes = {}    # key -> file suffix
    frame_size = {}  # key -> (H, W)
    frames_written_any = 0

    # 写入视频帧
    for t, step in enumerate(steps_ds):
        step_np = tf.nest.map_structure(lambda x: x.numpy(), step)
        obs = step_np.get("observation", {})
        try:
            frames = {
                "exterior_image_1_left": obs["exterior_image_1_left"],
                "exterior_image_2_left": obs["exterior_image_2_left"],
                "wrist_image_left":      obs["wrist_image_left"],
            }
        except KeyError as e:
            pbar.write(f"[WARN] {sample_id} step {t} missing {e}; frame skipped.")
            continue

        # 按需初始化 writer（每相机一次）
        for k, rgb in frames.items():
            if k not in writers:
                H, W = int(rgb.shape[0]), int(rgb.shape[1])
                frame_size[k] = (H, W)
                out_base = sample_dir / f"{k}"  # 不带后缀
                vw, suf = _make_writer(out_base, (H, W), fps=FPS, lossless=USE_LOSSLESS)
                writers[k] = vw
                suffixes[k] = suf

            bgr = cv2.cvtColor(frames[k], cv2.COLOR_RGB2BGR)
            writers[k].write(bgr)
            frames_written_any += 1

    # 关闭 writer
    for k, vw in writers.items():
        vw.release()

    if frames_written_any == 0:
        # 极端情况：虽然有文本，但所有帧都失败了；不删除 JSONL（通常仍然有用）
        pbar.write(f"[WARN] {sample_id}: no valid frames written; videos likely empty.")
    else:
        wrote_videos += 1

    pbar.update(1)

pbar.close()
jsonl_f.close()
print(f"Language JSONL saved to: {jsonl_path}")
print(f"Seen samples (IDs assigned): {seen_samples}")
print(f"Episodes with videos written: {wrote_videos}")
print(f"Output root: {OUTPUT_ROOT}")


Per-camera videos (only if any text):   0%|          | 1/20400 [00:01<8:06:07,  1.43s/ep]

[SKIP] id00001: all three instructions empty; no JSONL, no video.


Per-camera videos (only if any text):   0%|          | 2/20400 [00:02<8:11:02,  1.44s/ep]

[SKIP] id00002: all three instructions empty; no JSONL, no video.


Per-camera videos (only if any text):   0%|          | 3/20400 [00:03<6:15:14,  1.10s/ep]

...

[SKIP] id20395: all three instructions empty; no JSONL, no video.


Per-camera videos (only if any text): 100%|██████████| 20400/20400 [6:18:40<00:00,  1.18s/ep]

[SKIP] id20399: all three instructions empty; no JSONL, no video.
[SKIP] id20400: all three instructions empty; no JSONL, no video.
[INFO] Reached max_samples=20400 (by seen samples), stopping.


Per-camera videos (only if any text): 100%|██████████| 20400/20400 [6:18:42<00:00,  1.11s/ep]

Language JSONL saved to: F:\PangYe\shit4\language_instructions.jsonl
Seen samples (IDs assigned): 20401
Episodes with videos written: 11048
Output root: F:\PangYe\shit4


## only text

In [8]:
# JSONL 文件路径与写入模式（"a" 追加；如需重写用 "w"）
max_samples = 20000
JSONL_MODE = "a"
jsonl_path = OUTPUT_ROOT / "language_instructions.jsonl"
jsonl_path.parent.mkdir(parents=True, exist_ok=True)
jsonl_f = open(jsonl_path, JSONL_MODE, encoding="utf-8")

total_examples = info.splits[split].num_examples or None
processed_samples = 0
written_records = 0

pbar = tqdm(
    total=(max_samples if max_samples is not None else total_examples),
    desc="Collecting language instructions -> JSONL",
    unit="ep"
)

try:
    for sample_index, example in enumerate(ds, start=1):
        # 限制最多处理多少个样本
        if max_samples is not None and processed_samples >= max_samples:
            pbar.write(f"[INFO] Reached max_samples={max_samples}, stopping.")
            break

        # 取 steps 子数据集（应为 tf.data.Dataset）
        if "steps" not in example:
            pbar.write(f"[WARN] sample {sample_index} has no 'steps'; skipped.")
            continue
        steps_ds = example["steps"]
        if not isinstance(steps_ds, tf.data.Dataset):
            pbar.write(f"[WARN] sample {sample_index} 'steps' is not tf.data.Dataset; skipped.")
            continue

        # 样本 ID（沿用你的编号规则）
        sample_id = f"id{processed_samples + 1:0{SAMPLE_PAD}d}"

        # 从该样本的第一步中抓三条语言（若缺失则写空字符串）
        first_step_found = False
        for step in steps_ds.take(1):
            step_np = tf.nest.map_structure(lambda x: x.numpy(), step)
            rec = {
                "sample_id": sample_id,
                "instruction_1": decode_str(step_np.get("language_instruction", b"")),
                "instruction_2": decode_str(step_np.get("language_instruction_2", b"")),
                "instruction_3": decode_str(step_np.get("language_instruction_3", b"")),
            }
            jsonl_f.write(json.dumps(rec, ensure_ascii=False) + "\n")
            jsonl_f.flush()
            written_records += 1
            first_step_found = True

        if not first_step_found:
            pbar.write(f"[WARN] sample {sample_index} has 0 steps; no instruction written.")

        processed_samples += 1
        pbar.update(1)

finally:
    jsonl_f.close()
    pbar.close()

print(f"Done. Episodes processed: {processed_samples}")
print(f"Records written to JSONL: {written_records}")
print(f"Language JSONL path: {jsonl_path}")

[INFO] Reached max_samples=20000, stopping.


Done. Episodes processed: 20000
Records written to JSONL: 20000
Language JSONL path: F:\PangYe\text_in\language_instructions.jsonl
